# 1. split source reply ids

In [ ]:
def split_source_reply(txt_file):
  """
  txt_file: 'train.data.txt'
  """
  with open(txt_file) as f:
    ids = f.readlines()
  source_ids = []
  reply_ids = []
  source_txt_file = txt_file.split('.')[0] + '_source_data.txt'
  reply_txt_file = txt_file.split('.')[0] + '_reply_data.txt'
  for i in range(len(ids)):
    source_ids.append(ids[i].split(',')[0].strip())
    reply_ids.extend([r.strip() for r in ids[i].split(',')[1:]])
  # save source_ids
  with open(source_txt_file,'w') as f:
    for i in source_ids:
      f.write(i)
      f.write('\n')
  # save reply_ids
  with open(reply_txt_file,'w') as f:
    for i in reply_ids:
      f.write(i)
      f.write('\n')
# split_source_reply('dev.data.txt')

# 2. Crawl tweet

In [ ]:
# !twarc2 hydrate  dev.txt > dev_reply_data.jsonl
# resource: https://scholarslab.github.io/learn-twarc/06-twarc-command-basics.html

# 3. Get features

In [ ]:
from collections import defaultdict
import json
import pandas as pd
def filter_text_reply_count(jsonl_file_name):
    """
    jsonl_file_name: 'dev_source_data.jsonl'
    """
    json_file_name = '_'.join(jsonl_file_name.split('_')[:-1]) + '.json'
    json_data = pd.read_json(path_or_buf=jsonl_file_name, lines=True)
    data_dict = defaultdict(dict)
    for i in range(json_data.shape[0]):
        for j in range(len(json_data.data.iloc[i])):
            data_dict[json_data.data.iloc[i][j]['id']]['text'] = json_data.data.iloc[i][j]['text']
            data_dict[json_data.data.iloc[i][j]['id']]['reply_count'] = json_data.data.iloc[i][j]['public_metrics']['reply_count']
            data_dict[json_data.data.iloc[i][j]['id']]['like_count'] = json_data.data.iloc[i][j]['public_metrics']['like_count']
            data_dict[json_data.data.iloc[i][j]['id']]['retweet_count'] = json_data.data.iloc[i][j]['public_metrics']['retweet_count']
            data_dict[json_data.data.iloc[i][j]['id']]['quote_count'] = json_data.data.iloc[i][j]['public_metrics']['quote_count']
            data_dict[json_data.data.iloc[i][j]['id']]['possibly_sensitive'] = json_data.data.iloc[i][j]['possibly_sensitive']
            
    #  convert into json format
    dict_json=json.dumps(data_dict)
    # save json file
    with open(json_file_name, 'w+') as file:
        file.write(dict_json)
# filter_text_reply_count('dev_reply_data.jsonl')

# 4. Convert json to dataframe

In [ ]:
def json2df(json_file):
    """
    json_file: 'train_reply.json'
    """
    with open(json_file,'r+') as file:
        content=file.read()
    content=json.loads(content)#将json格式文件转化为python的字典文件
    df = pd.DataFrame(content)
    df = df.stack()
    df = df.unstack(0)
    return df
# df = json2df('train_reply.json')